# Packages/Imports

In [1]:
from tinydb import TinyDB, Query

# Helper Functions

In [ ]:
# Takes in a player level or creature CR and outputs the corresponding proviciency modifier.
def profByLevel(level):
    return 2 + int((level - 1) / 4)

# Global Data

In [4]:
damageType = [
    "Piercing",
    "Bludgeoning",
    "Slashing",
    "Cold",
    "Fire",
    "Lightning",
    "Thunder",
    "Poison",
    "Acid",
    "Necrotic",
    "Radiant",
    "Force",
    "Psychic"
]

alignments = [
    "Unaligned",
    "Lawful Good",
    "Lawful Neutral",
    "Lawful Evil",
    "Neutral Good",
    "True Neutral",
    "Neutral Evil",
    "Chaotic Good",
    "Chaotic Neutral",
    "Chaotic Evil"
    ]

# Database Files

In [ ]:
# players = TinyDB('DBFiles/players.json')
# creatures = TinyDB('DBFiles/creatures.json')
# sizes = TinyDB('DBFiles/size.json')
# types = TinyDB('DBFiles/type.json')
# conditions = TinyDB('DBFiles/condition.json')
# classes = TinyDB('DBFiles/class.json')
# species = TinyDB('DBFiles/species.json')
# skills = TinyDB('DBFiles/skill.json')
# senses = TinyDB('DBFiles/senses.json')
# parties = TinyDB('DBFiles/party.json')

# Core Object Definitions

# Testing Cells

In [5]:
# Test to determine Proficiency bonus by Challenge Rating
cr = [0, 0.125, 0.25, 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17 ,18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]

for i in cr:
    print(f"{i}: {profByLevel(i)}")

0: 2
0.125: 2
0.25: 2
0.5: 2
1: 2
2: 2
3: 2
4: 2
5: 3
6: 3
7: 3
8: 3
9: 4
10: 4
11: 4
12: 4
13: 5
14: 5
15: 5
16: 5
17: 6
18: 6
19: 6
20: 6
21: 7
22: 7
23: 7
24: 7
25: 8
26: 8
27: 8
28: 8
29: 9
30: 9
